__PID controller / compensator designer based on 2nd order approximation 

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib notebook
# %pylab widgetsnbextension
import matplotlib.pyplot as plt
from numpy import log as ln, arange, roots
from math import atan, sqrt, pi, degrees, tan, e
# from sympy import symbols, simplify, fraction,solve, pprint, evalf, im, I
from math import isclose
from control import root_locus, tf, tfdata, feedback
import sympy as S



__Run the code below, and you will get an interactive window below to get the Overshoot from the damping ratio and vice versa

In [8]:

def angle_from_damping_ratio(zeta):
    '''the angle needed to meet the zeta condition, drawn from origin to + pole, and by symmetry can be extended to bottom angle
    taken by drawing a line from the origin to the complex conjugate pole ie the -pole'''
#     try:
#         zeta = float(zeta)
#     except:
#         print("Invalid zeta")
#         return 
    if zeta == 1:
        return 0
    elif abs(zeta) == 0:
        return 90
    x = sqrt(1-zeta**2)/zeta
    angle = atan(x)
    return degrees(angle)

def os_from_damping_ratio(zeta):
    num = -pi * zeta
    den = sqrt(1-zeta**2)
    return 100 * e**num/den

def damping_ratio_from_os(os):
    os = os/100
    zeta = -ln(os)/sqrt(pi**2+(ln(os))**2)
    return zeta
def show_zeta(zeta):
    if zeta:
        zeta = float(zeta)
    else:
        zeta = 0
    if zeta>1 or zeta <0:
        return "Zeta is between 0 and 1"
    global_zeta = str(zeta)
    os = os_from_damping_ratio(zeta)
    ang = angle_from_damping_ratio(zeta)
    out = {"Zeta":zeta, "OS":os, "Angle deg":ang}
    print(out)

def show_os(os):
    if os:
        os = float(os)
    else:
        os = 1
    zeta = damping_ratio_from_os(os)
    ang = angle_from_damping_ratio(zeta)
    out = {"Zeta":zeta, "OS":os, "Angle deg":ang}
    print(out)

    
interact(show_zeta, zeta = "0")
interact(show_os, os = "1")

interactive(children=(Text(value='0', description='zeta'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Text(value='1', description='os'), Output()), _dom_classes=('widget-interact',))

<function __main__.show_os(os)>

__Find PD Gain and zero location__ Give it an open loop transfer function, and the desired pole location. It will print the KD (deriviative gain), and the zero location in the s-plane. Run the cell right below then below that there's a cell with an example,

In [4]:
def find_zero_and_gain_for_PD_controller(oltf, pole, rounding_accuracy=3):
    ''' takes open loop transfer func,oltf, and desired pole. returns the zero location, as well as
     the gain of the PD controller, to meet that desired pole requirement. Assumes pole provided is in the 2nd quadrant, ie, left hand side of the JW axis or on it '''
    #sanitize the pole to make sure it is on the top of the LHS ie 2nd quadrant
    pole = ((-1* abs(pole.real)) + abs(pole.imag)*1j)
    oltf_at_pole = oltf.horner(pole)
    zero_location_angle = pi - angle(oltf_at_pole)
    sigma = abs(pole.real)
    wd = abs(pole.imag)
    zero_loction = sigma + wd/tan(zero_location_angle)
    kd = abs(1/( oltf_at_pole * (pole+zero_loction) ) )
    print({"kd": round(kd, rounding_accuracy), "zero":round(zero_loction, rounding_accuracy) })

In [5]:
#Example
# s = tf('s')
#oltf = 1/(s**2 + 4)
# desired_pole = (x,y*j)
# find_zero_and_gain_for_PD_controller(oltf, desired_pole)

In [6]:
def find_routh_poly_state_space(a):
    #extracts the polynomial from a state space
    #det(sI-A) = the poly that u use for routh hurwitz table from state space
    a = S.Matrix(a)
    e = S.eye(a.rows)
    x = S.symbols('s')
    e = e*x
    poly = S.expand(S.det(e - a))
    print(poly)


In [7]:
a = [[0, 3, 1], [2, 8, 1], [-10, -5, -2]]
find_routh_poly_state_space(a)

s**3 - 6*s**2 - 7*s - 52
